![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

# PostgreSQL for Python Developers

## Refactoring airline tweets database

In this project, we will use the PostgreSQL data definition language to optimize the airline tweets that we moved into PostgreSQL in an earlier project.

You will need access to a PostgreSQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See `https://hub.docker.com/_/postgres` for details on that option.  Unless you have a specific need to work with an existing installation, choosing a PostgreSQL version of 12 or higher is best.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Refactoring a database (enumerations)**

The table describing airline tweets was created in a prior project.  Your solution may not be identical to the sample solution, but most of these issues are likely to apply.

A number of columns were probably created as CHAR or VARCHAR field, but have only a small number of distinct values.  Those would be more efficiently and descriptively stored as enumerations.  Enumerations are discussed in somewhat more detail in the next lesson, but in brief, you can create one within a particular database using code similar to:

```python
cur.execute("CREATE TYPE mood AS ENUM ('sad', 'ok', 'happy');")
```

Such an enumeration would be useful, for example, for `airline_sentiment`:

```sql
ine=# SELECT DISTINCT airline_sentiment FROM tweets;
 airline_sentiment
-------------------
 negative
 positive
 neutral
(3 rows)
```

Create enumerated types for every column where it is relevant, and update the table `tweets` appropriately.

**Solution**

In [1]:
import psycopg2
user, pwd = 'ine_student', 'ine-password'
host, port = 'localhost', '5432'
conn = psycopg2.connect(database='ine', host=host, user=user, password=pwd, port=port)
cur = conn.cursor()

If you don't have the database already created, create it:

In [ ]:
# create PostgreSQL table
sql_create_tweets = '''
CREATE TABLE IF NOT EXISTS Tweets (
    tweet_id DECIMAL(18) PRIMARY KEY,
    airline_sentiment TEXT,
    airline_sentiment_confidence REAL,
    negativereason TEXT,
    negativereason_confidence REAL,
    airline TEXT,
    airline_sentiment_gold TEXT,
    name TEXT,
    negativereason_gold TEXT,
    retweet_count INT,
    text TEXT,
    tweet_coord TEXT,
    tweet_created TIMESTAMP WITH TIME ZONE,
    tweet_location TEXT,
    user_timezone TEXT
    );
'''
cur_dest.execute('DROP TABLE Tweets')
cur_dest.execute(sql_create_tweets)
con_dest.commit()

In [ ]:
con_src = sqlite3.connect('data/Airline-Tweets.sqlite') 
cur_src = con_src.cursor()
cur_src.execute("SELECT * FROM Tweets")

In [ ]:
sql_insert = """
INSERT INTO Tweets 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
for src_row in cur_src:
    row = tuple(data or None for data in src_row)
    cur_dest.execute(sql_insert, row)
    
con_dest.commit()

As a first step, we should find out all columns that have a small number of distinct values.


In [2]:
text_cols = []
cur.execute("SELECT * FROM tweets LIMIT 1;")
for col in cur.description:
    if col.type_code == 25:   # TEXT, adjust if your schema differs
        text_cols.append(col.name)

In [3]:
categories = dict()
for col in text_cols:
    cur.execute(f"SELECT count(DISTINCT {col}) FROM tweets;")
    num = cur.fetchone()[0]
    print(f"{col} has {num} distinct values")
    if num < 10:
        cur.execute(f"SELECT DISTINCT {col} FROM tweets;")
        categories[col] = [t[0] for t in cur]

airline_sentiment has 3 distinct values
negativereason has 11 distinct values
airline has 6 distinct values
airline_sentiment_gold has 4 distinct values
name has 5621 distinct values
negativereason_gold has 14 distinct values
text has 10413 distinct values
tweet_coord has 614 distinct values
tweet_location has 2227 distinct values
user_timezone has 74 distinct values


In [4]:
categories

{'airline_sentiment': ['negative', 'positive', 'neutral'],
 'airline': ['Virgin America',
  'Southwest',
  'Delta',
  'American',
  'US Airways',
  'United'],
 'airline_sentiment_gold': ['negative', 'positive', '', 'neutral']}

My initial conversion had an empty string for sentiment, which feels like a data problem. I tranformed it to a NULL.  Your initial conversion may not have had this problem.

In [5]:
cur.execute("UPDATE tweets SET airline_sentiment_gold = NULL WHERE airline_sentiment_gold = '';")
cur.execute("SELECT DISTINCT airline_sentiment_gold FROM tweets;")
cur.fetchall()

[(None,), ('negative',), ('positive',), ('neutral',)]

In [6]:
sql_sentiment = "CREATE TYPE sentiment AS ENUM ('negative', 'positive', 'neutral');"
sql_airline = """
CREATE TYPE airline 
AS ENUM ('Virgin America', 'Southwest',  'Delta', 'American',  'US Airways', 'United');
"""
cur.execute(sql_sentiment)
cur.execute(sql_airline)

In [7]:
sql_enum_sentiment = """
ALTER TABLE tweets 
ALTER COLUMN airline_sentiment 
TYPE sentiment
USING airline_sentiment::sentiment;
"""
sql_enum_sentiment_gold = """
ALTER TABLE tweets 
ALTER COLUMN airline_sentiment_gold
TYPE sentiment
USING airline_sentiment_gold::sentiment;
"""
sql_airlines = """
ALTER TABLE tweets
ALTER COLUMN airline
TYPE airline
USING airline::airline;
"""
cur.execute(sql_enum_sentiment)
cur.execute(sql_enum_sentiment_gold)
cur.execute(sql_airlines)

In [8]:
cur.execute("SELECT * FROM tweets LIMIT 1;")
cur.description

(Column(name='tweet_id', type_code=1700),
 Column(name='airline_sentiment', type_code=36580),
 Column(name='airline_sentiment_confidence', type_code=700),
 Column(name='negativereason', type_code=25),
 Column(name='negativereason_confidence', type_code=700),
 Column(name='airline', type_code=36588),
 Column(name='airline_sentiment_gold', type_code=36580),
 Column(name='name', type_code=25),
 Column(name='negativereason_gold', type_code=25),
 Column(name='retweet_count', type_code=23),
 Column(name='text', type_code=25),
 Column(name='tweet_coord', type_code=25),
 Column(name='tweet_created', type_code=1184),
 Column(name='tweet_location', type_code=25),
 Column(name='user_timezone', type_code=25))

In [9]:
# Normally would commit here, for project I restore the original state
conn.rollback()

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Adding foreign keys**

The columns `user_timezone` and `airline` feel like entities about which we would like to be able to store additional information.  For example, what is the UTC offset and daylight savings change schedule at a given timezone?  What is the contact information for a given airline, and what airports do they service?

The this task you need to complete two parts.  First you should create tables `timezones` and `airlines`, and populate them with the values currently existing for the foreign key column.  In each case, simply add a second text column called `description` (although more specific fields would clearly be needed for a production use).

Second, create a foreign key constraint between the `tweets` table and these new tables that provide information about the entitities.

**Solution**

In [10]:
sql_airlines = """
CREATE TABLE airlines (
    airline TEXT PRIMARY KEY,
    description TEXT DEFAULT NULL
    );
"""
sql_timezones = """
CREATE TABLE timezones (
    timezone TEXT PRIMARY KEY,
    description TEXT DEFAULT NULL
    );
"""
cur.execute(sql_airlines)
cur.execute(sql_timezones)

In [11]:
sql_populate_airlines = """
INSERT INTO airlines (airline)
SELECT DISTINCT airline FROM tweets;
"""
sql_populate_timezones = """
INSERT INTO timezones (timezone)
SELECT DISTINCT user_timezone FROM tweets;
"""
cur.execute(sql_populate_airlines)
cur.execute(sql_populate_timezones)

Now we need to add the foreign key.

In [12]:
sql_airline_constraint = """
ALTER TABLE tweets
ADD FOREIGN KEY (airline)
REFERENCES airlines
ON DELETE CASCADE;
"""
cur.execute(sql_airline_constraint)

In [13]:
sql_timezone_constraint = """
ALTER TABLE tweets
ADD FOREIGN KEY (user_timezone)
REFERENCES timezones(timezone)
ON DELETE SET NULL;
"""
cur.execute(sql_timezone_constraint)

In [14]:
# Normally would commit here, for project I restore the original state
conn.rollback()

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)